In [4]:
import time
import requests


class AutoJWTClient:
    def __init__(self, base_url, login_url, email, password):
        self.base_url = base_url.rstrip("/")
        self.login_url = login_url
        self.email = email
        self.password = password
        self.access_token = None
        self.refresh_token = None
        self.token_expiry = 0  # epoch time

        self.session = requests.Session()
        self._login()

    def _login(self):
        response = self.session.post(
            f"{self.base_url}{self.login_url}",
            json={"email": self.email, "password": self.password}
        )
        if response.status_code != 200:
            raise Exception("Login failed")

        data = response.json()
        self.access_token = data["access"]
        self.refresh_token = data.get("refresh")
        self.token_expiry = self._decode_expiry(self.access_token)

    def _decode_expiry(self, token):
        import base64
        import json

        try:
            payload = token.split('.')[1]
            padding = '=' * (-len(payload) % 4)
            decoded = base64.urlsafe_b64decode(payload + padding)
            exp = json.loads(decoded)["exp"]
            return exp
        except Exception:
            return 0

    def _refresh_token_if_needed(self):
        now = time.time()
        if now >= self.token_expiry - 10:  # 10s buffer
            if not self.refresh_token:
                self._login()
            else:
                response = self.session.post(
                    f"{self.base_url}/api/token/refresh/",
                    json={"refresh": self.refresh_token}
                )
                if response.status_code != 200:
                    self._login()
                else:
                    data = response.json()
                    self.access_token = data["access"]
                    self.token_expiry = self._decode_expiry(self.access_token)

    def request(self, method, path, **kwargs):
        self._refresh_token_if_needed()

        headers = kwargs.pop("headers", {})
        headers["Authorization"] = f"Bearer {self.access_token}"
        kwargs["headers"] = headers

        url = f"{self.base_url}{path}"
        return self.session.request(method, url, **kwargs)

    def get(self, path, **kwargs):
        return self.request("GET", path, **kwargs)

    def post(self, path, **kwargs):
        return self.request("POST", path, **kwargs)

    def put(self, path, **kwargs):
        return self.request("PUT", path, **kwargs)

    def delete(self, path, **kwargs):
        return self.request("DELETE", path, **kwargs)


In [9]:
client = AutoJWTClient(
    base_url="http://192.168.123.97:8000/api",
    login_url="/auth/token/",
    email="f4rukseker@gmail.com",
    password="pars"
)

In [35]:
from pprint import pprint

## ilaçları görüntülemek için

In [43]:
response = client.get('/medication/medications/')

In [44]:
pprint(response.json())

[{'default_dose_amount': '1.00',
  'default_dose_unit': 'mg',
  'id': 1,
  'is_active': True,
  'name': 'salofalk',
  'schedules': [{'created_at': '2025-05-13T18:59:04.331789',
                 'day_of_month': None,
                 'days_of_week': [1, 2, 3, 4, 5, 6, 7],
                 'dose_amount': '2.00',
                 'dose_unit': 'mg',
                 'doses_per_period': 3,
                 'end_date': '2025-05-20',
                 'frequency': 'daily',
                 'id': 1,
                 'interval': 2,
                 'medication': 1,
                 'start_date': '2025-05-13',
                 'updated_at': '2025-05-13T18:59:04.331789'}]},
 {'default_dose_amount': '12.00',
  'default_dose_unit': 'mg',
  'id': 2,
  'is_active': True,
  'name': 'Şeker',
  'schedules': []}]


## Spesifik bir ilacı görüntülemek için

In [50]:
medication_id: int = 1
response = client.get(f'/medication/medications/{medication_id}/')
response.text

'{"id":1,"name":"salofalk","is_active":true,"default_dose_amount":"1.00","default_dose_unit":"mg","schedules":[{"id":1,"start_date":"2025-05-13","end_date":"2025-05-20","frequency":"daily","interval":2,"day_of_month":null,"doses_per_period":3,"dose_amount":"2.00","dose_unit":"mg","created_at":"2025-05-13T18:59:04.331789+03:00","updated_at":"2025-05-13T18:59:04.331789+03:00","medication":1,"days_of_week":[1,2,3,4,5,6,7]}]}'

## planlama

In [51]:
response = client.get('/medication/medication-schedules')
response.text

'[{"id":1,"start_date":"2025-05-13","end_date":"2025-05-20","frequency":"daily","interval":2,"day_of_month":null,"doses_per_period":3,"dose_amount":"2.00","dose_unit":"mg","created_at":"2025-05-13T18:59:04.331789+03:00","updated_at":"2025-05-13T18:59:04.331789+03:00","medication":1,"days_of_week":[1,2,3,4,5,6,7]}]'

## Bugün hangi ilaçları alıcak

In [56]:

response = client.get('/medication/today')
response.text

'[{"medication_name":"salofalk","dose":2.0,"unit":"mg","schedules_id":1}]'